In [62]:
import pandas as pd
import numpy as np
from pprint import pprint

In [81]:
import fitz

In [82]:
# mypdf = open(r'C:\Users\aritr\OneDrive\Documents\un_habitat\Turkey_Eleventh_Development_Plan_2019-2023.pdf',mode='rb')
# pdf_doc = PyPDF2.PdfFileReader(mypdf)

# print('Number of Pages: ', pdf_doc.numPages)
filepath = r'C:\Users\aritr\OneDrive\Documents\un_habitat\npr_analyses_pre\Kingdom of Bahrain NUA Report 2021-compressed.pdf'

#text = ''
text = []

with fitz.open(filepath) as doc:
    for page in doc:
        #text += page.getText()
        text.append(page.getText())
text

Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.
Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.
Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.
Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.
Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.
Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.
Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.
Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.
Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.
Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.
Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.
Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.
Deprecation: 'getText' removed from class 'Page' aft

[' \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nNational report of the Kingdom of \nBahrain \n \nProgress in the implementation of the new urban \nplan NUA \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n',
 ' \n2021 \neven\nts \n',
 ' \n \n \n \n \nContributors to the report \n \n \n \nMinistries \n \noffice of the prime minister \n \n- Cabinet Affairs Ministry \n \n- Foreign Ministry \n \n- Interior Ministry \n \n- Ministry of Finance and National Economy \n \n- Housing Ministry \n \n-Ministry of Works and Municipal Affairs \n \nand urban planning \n \n- Ministry of Communications \n \n- Ministry of Labor and Social Development \n \n- Ministry of Industry, Trade and Tourism \n \n- Ministry of Health \n \n- \nMinistry of Justice, \nIslamic Affairs and \nEndowments \n- Ministry of Education \n \n- Ministry of Youth and Sports Affairs \n  \nbodies \n \n- Electronic Information and Government Authority \n \n- 

In [83]:
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aritr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [84]:
from sklearn.feature_extraction.text import CountVectorizer
import re

import spacy

In [85]:
import gensim 
from gensim import corpora
from gensim import models
from gensim.models import CoherenceModel

In [86]:
stop_words = stopwords.words('english')
#avoid_words = ['national', 'plan', 'turkey', 'percent','development','policy', 'bangladesh', 'iraq', 'trinidad', 'tobago', 'sierra', 'use', 'leone', 'policies', 'service','-', 'increase', 'ensure', 'sector','develop', 'country', 'order', 'support','strategy']
avoid_words = pd.read_csv(r'C:\Users\aritr\OneDrive\Documents\un_habitat\stopwords.txt', names = ['stopwords'])
stop_words.extend([i.lower() for i in list(avoid_words.stopwords)])
pattern = "(?u)\\b[\\w-]+\\b"

In [87]:
def preprocess_text(word):
    word = word.lower()
    word = re.sub(r'\d+', '', word)
    return word

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, forms_allowed):
    texts_ = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_.append([token.lemma_ for token in doc if token.pos_ in forms_allowed])
        return texts_

In [88]:
vectorizer = CountVectorizer(stop_words = 'english', token_pattern=pattern, preprocessor = preprocess_text)
analyze = vectorizer.build_analyzer()
clean1 = [analyze(i) for i in text]
clean1

[['national',
  'report',
  'kingdom',
  'bahrain',
  'progress',
  'implementation',
  'new',
  'urban',
  'plan',
  'nua'],
 ['ts'],
 ['contributors',
  'report',
  'ministries',
  'office',
  'prime',
  'minister',
  'cabinet',
  'affairs',
  'ministry',
  'foreign',
  'ministry',
  'interior',
  'ministry',
  'ministry',
  'finance',
  'national',
  'economy',
  'housing',
  'ministry',
  'ministry',
  'works',
  'municipal',
  'affairs',
  'urban',
  'planning',
  'ministry',
  'communications',
  'ministry',
  'labor',
  'social',
  'development',
  'ministry',
  'industry',
  'trade',
  'tourism',
  'ministry',
  'health',
  'ministry',
  'justice',
  'islamic',
  'affairs',
  'endowments',
  'ministry',
  'education',
  'ministry',
  'youth',
  'sports',
  'affairs',
  'bodies',
  'electronic',
  'information',
  'government',
  'authority',
  'electricity',
  'water',
  'authority',
  'sustainable',
  'energy',
  'authority',
  'national',
  'oil',
  'gas',
  'authority',
  'u

In [89]:
bigram = gensim.models.Phrases(clean1, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[clean1], threshold = 100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

clean_bigrams = make_bigrams(clean1)
#print(len(clean_bigrams))
nlp = spacy.load('en_core_web_sm', disable = ['parser', 'ner'])

clean_lemmatized = [lemmatization([i], forms_allowed  = ['NOUN','ADJ','ADV', 'VERB']) for i in clean_bigrams]
#print(clean_lemmatized)

#second stage
clean2 = []
for i in clean_lemmatized:
    clean2.append([word for word in i[0] if word not in stop_words])
    
#clean2 = [word for i in clean_lemmatized if word not in stop_words for word in i]
print(clean2)

[['kingdom', 'progress_implementation', 'urban'], ['ts'], ['contributor', 'office', 'cabinet', 'finance', 'affair', 'urban', 'labor', 'tourism', 'endowment', 'education', 'youth', 'sport', 'affair', 'body', 'electronic', 'government', 'electricity', 'water', 'energy', 'oil', 'gas', 'urban', 'work', 'box', 'enable', 'council', 'woman', 'economic', 'board', 'environment', 'education', 'council', 'civil', 'center', 'study', 'university', 'microfinance', 'bank', 'office', 'resident', 'coordinator', 'kingdom', 'office', 'undp', 'resident', 'representative', 'kingdom', 'united_nation', 'human_settlement', 'office', 'kuwait', 'united_nation', 'human_settlement', 'habitat'], ['kingdom'], ['royal', 'highness', 'forgive', 'revere'], ['open', 'word', 'kingdom', 'land', 'glory', 'civilization', 'ancient', 'time', 'authentic', 'inheritance', 'inherit', 'proud', 'continue', 'consolidate', 'approach', 'make', 'good', 'seek', 'people', 'today', 'partner', 'sake', 'humanity', 'ambition', 'limited', 'ex

In [90]:
from collections import defaultdict

allwords = [word for page in clean2 for word in page]
frequency = defaultdict(int)
for token in allwords:
    frequency[token] += 1

#texts = [i for i in allwords if frequency[i] > 1]
#pprint(texts)
sorted(frequency.items(), key=lambda kv: kv[1], reverse = True)

[('kingdom', 280),
 ('urban', 276),
 ('government', 191),
 ('housing', 158),
 ('energy', 150),
 ('woman', 119),
 ('pm', 93),
 ('land', 92),
 ('economic', 90),
 ('water', 89),
 ('initiative', 88),
 ('public', 81),
 ('various', 80),
 ('social', 74),
 ('work', 65),
 ('building', 62),
 ('progress_implementation', 59),
 ('effort', 59),
 ('city', 59),
 ('important', 58),
 ('environment', 57),
 ('resource', 57),
 ('addition', 57),
 ('population', 57),
 ('contribute', 56),
 ('training', 55),
 ('aim', 53),
 ('launch', 53),
 ('local', 49),
 ('indicator', 47),
 ('promote', 46),
 ('benefit', 45),
 ('center', 44),
 ('participation', 44),
 ('business', 43),
 ('financial', 43),
 ('international', 41),
 ('construction', 41),
 ('time', 40),
 ('infrastructure', 40),
 ('citizen', 40),
 ('consumption', 39),
 ('waste', 39),
 ('renewable', 39),
 ('integrate', 38),
 ('adopt', 38),
 ('field', 37),
 ('standard', 37),
 ('efficiency', 37),
 ('road', 37),
 ('environmental', 37),
 ('make', 36),
 ('cost', 36),
 ('c

In [91]:
#texts 
#texts = [texts]
texts = clean2
id2word = corpora.Dictionary(texts)
num_topics = 10
id2word.filter_extremes(no_below = 1, no_above = .5)
corpus = [id2word.doc2bow(i) for i in texts]

In [92]:
lda_model=gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                          id2word=id2word, 
                                          num_topics= num_topics, 
                                          random_state= 567, 
                                          update_every=1, 
                                          chunksize=100, 
                                          passes=10, 
                                          alpha='auto', 
                                          per_word_topics=True)

In [93]:
pprint(lda_model.print_topics())
#doc_lda=lda_model[corpus]

[(0,
  '0.036*"government" + 0.025*"housing" + 0.014*"vehicle" + 0.014*"cost" + '
  '0.014*"company" + 0.013*"efficiency" + 0.013*"ton" + 0.012*"revenue" + '
  '0.012*"water" + 0.011*"governorate"'),
 (1,
  '0.036*"government" + 0.016*"housing" + 0.015*"electronic" + '
  '0.013*"heritage" + 0.012*"center" + 0.012*"building" + 0.011*"house" + '
  '0.010*"urban" + 0.010*"economic" + 0.010*"social"'),
 (2,
  '0.039*"woman" + 0.019*"pm" + 0.015*"participation" + 0.012*"initiative" + '
  '0.012*"economic" + 0.011*"government" + 0.010*"business" + 0.010*"position" '
  '+ 0.009*"indicator" + 0.008*"aim"'),
 (3,
  '0.016*"effort" + 0.010*"enterprise" + 0.010*"local" + 0.009*"housing" + '
  '0.009*"small" + 0.009*"measure" + 0.009*"innovation" + '
  '0.009*"communication" + 0.008*"public" + 0.008*"transport"'),
 (4,
  '0.033*"import" + 0.020*"pm" + 0.012*"interface" + 0.011*"contain" + '
  '0.010*"justice" + 0.010*"propose" + 0.008*"prevent" + 0.008*"exhaust" + '
  '0.007*"organism" + 0.007*"av

In [94]:
top_topics = lda_model.top_topics(corpus)

In [95]:
avg_topic_coherence = sum(t[1] for t in top_topics)/num_topics
print('Average topic coherence : ',  avg_topic_coherence)

Average topic coherence :  -5.518210177965047


In [96]:
pprint(top_topics)

[([(0.052944392, 'energy'),
   (0.021662712, 'water'),
   (0.01720926, 'waste'),
   (0.016583785, 'urban'),
   (0.013757539, 'renewable'),
   (0.013198566, 'resource'),
   (0.012743507, 'consumption'),
   (0.011511371, 'environmental'),
   (0.010848541, 'land'),
   (0.01067999, 'solar'),
   (0.009987936, 'efficiency'),
   (0.009191242, 'building'),
   (0.008797356, 'environment'),
   (0.008376989, 'population'),
   (0.0078419745, 'city'),
   (0.0077832523, 'electricity'),
   (0.007728212, 'adopt'),
   (0.0076582883, 'production'),
   (0.0075626182, 'sea'),
   (0.0062878258, 'source')],
  -1.5736393812736489),
 ([(0.015865987, 'effort'),
   (0.010305087, 'enterprise'),
   (0.009715066, 'local'),
   (0.009328543, 'housing'),
   (0.009208664, 'small'),
   (0.0091799, 'measure'),
   (0.008998204, 'innovation'),
   (0.00861766, 'communication'),
   (0.008263974, 'public'),
   (0.008115431, 'transport'),
   (0.007943149, 'technology'),
   (0.00750392, 'infrastructure'),
   (0.007329628, 'esp

In [97]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

In [98]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(



PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.059964  0.201064       1        1  20.635113
8     -0.258902 -0.158729       2        1  20.201096
3     -0.068497  0.108088       3        1  14.962642
6     -0.063030  0.108682       4        1  12.499135
1      0.017845  0.057528       5        1  10.116046
0     -0.078260  0.026472       6        1   7.689995
5      0.057341 -0.021029       7        1   6.141401
7      0.169812 -0.101722       8        1   5.394290
9      0.109949 -0.133931       9        1   1.661825
4      0.053779 -0.086423      10        1   0.698457, topic_info=                         Term        Freq       Total Category  logprob  \
1                       urban  245.000000  245.000000  Default  30.0000   
20                     energy  153.000000  153.000000  Default  29.0000   
41                      woman  117.000000  117.000000  Default  28.0000   
93                         pm   77.000000   77.000000  Default  27.0000   
0     progress_implementation   48.000000   48.000000  Default  26.0000   
...                       ...         ...         ...      ...      ...   
152                technology    0.585296   39.590616  Topic10  -5.1240   
551                protection    0.359923   13.863834  Topic10  -5.6103   
1833               profession    0.310506    3.709537  Topic10  -5.7579   
404                      city    0.318377   52.341618  Topic10  -5.7329   
1992            concentration    0.270962    1.687253  Topic10  -5.8942   

      loglift  
1     30.0000  
20    29.0000  
41    28.0000  
93    27.0000  
0     26.0000  
...       ...  
152    0.7498  
551    1.3129  
1833   2.4836  
404   -0.1383  
1992   3.1352  

[547 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
498       7  0.727420    ability
498       8  0.181855    ability
96        1  0.103104       able
96        4  0.309311       able
96        7  0.515518       able
...     ...       ...        ...
1723      8  0.878942   workshop
517       7  0.677305  worldwide
43        1  0.932247      youth
43        4  0.046612      youth
2136      4  0.956917       zone

[860 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 9, 4, 7, 2, 1, 6, 8, 10, 5])